In [0]:
# Imports
import pandas as pd

In [0]:
''' laying out the database scheme and generating sample data to use for the models

TABLE 1: Customers
COLUMNS:
- Id
- First Name
- Last Name
- Age
- Location

TABLE 2: Bank
- Customer ID
- Annual Income
- Debt-To-Income Ratio (DTI)
- Loan-to-Value Ratio (LTV) 
- Average Monthly Spending 
- Credit Score

TABLE 2: Transactions 
COLUMNS: 
- Customer ID 
- Transaction Date
- Amount 
- Recipient
- Device Type
'''

In [0]:
# sample data for customers table
customers = {
    'id': [1, 2, 3, 4, 5],
    'first_name': ['Mitch', 'Auston', 'William', 'Joseph', 'John'],
    'last_name' : ['Marner', 'Matthews', 'Nylander', 'Woll', 'Tavares'],
    'age': [28, 27, 29, 26, 34],
    'location': ['CA', 'USA', 'CA', 'USA', 'Canada']
}

customers_df = pd.DataFrame(customers)

In [0]:
# Create database and tables

database_name = "banking_database"
customers_table = "customers"
transaction_table = "transactions"
bank_table = "bank"

# Convert pandas DataFrame to Spark DataFrame
spark_df = spark.createDataFrame(customers_df)

spark.sql(f"CREATE DATABASE IF NOT EXISTS `{database_name}`")

# Write the Spark DataFrame to a Delta table
# using .write.saveAsTable()
# 'mode("overwrite")' will replace the table if it already exists.
# Use 'mode("append")' to add data to an existing table.
# Use 'mode("errorifexists")' (default) to throw an error if table exists.
spark_df.write.format("delta").mode("overwrite").saveAsTable(f"{database_name}.{customers_table}")

print(f"\nManaged Delta table '{database_name}.{customers_table}' created successfully!")



